# Understanding IfcRevolvedAreaSolid



In [ ]:
from dataclasses import dataclass

import numpy as np

import ada
from ada.config import Settings
from ada.core.constants import X, Y, Z, O
from ada.core.curve_utils import get_center_from_3_points_and_radius, CurveData
from ada.core.vector_utils import unit_vector, normal_to_points_in_plane, transform3d, EquationOfPlane, calc_yvec, Plane
from ada.occ.utils import make_eq_plane_object, make_ori_vector

Settings.make_param_elbows = True

In [ ]:
def pipe_w_multiple_bends() -> ada.Assembly:
    z = 3.2
    y0 = -200e-3
    x0 = -y0
    coords = [
        (0, y0, z),
        (5 + x0, y0, z),
        (5 + x0, y0 + 5, z),
        (10, y0 + 5, z + 2),
        (10, y0 + 5, z + 10),
    ]
    pipe1 = ada.Pipe(
        "Pipe1",
        coords,
        ada.Section("PSec", "PIPE", r=0.10, wt=5e-3),
    )

    return ada.Assembly('TestModel') / ada.Part('TestSite') /  pipe1

In [ ]:
@dataclass
class SegData:
    eqp: EquationOfPlane
    extr_start_point: np.ndarray
    arc_curve: CurveData
    abs_coord: np.ndarray


def evaluate_segment(pipe_elbow: ada.PipeSegElbow, plane_name: str, make_revolve_p_as_vector=False):
    xvec1 = unit_vector(pipe_elbow.xvec1)

    p1, p2, p3 = pipe_elbow.p1.p, pipe_elbow.p2.p, pipe_elbow.p3.p
    normal = normal_to_points_in_plane([p1, p2, p3])

    cd = get_center_from_3_points_and_radius(p1, p2, p3, pipe_elbow.bend_radius)
    extrusion_start_p = pipe_elbow.arc_seg.p1
    delta = cd.center - extrusion_start_p

    yvec = calc_yvec(xvec1, normal)
    global_csys = (X, Y, Z)
    new_csys = (normal, yvec, xvec1)

    n_tra, diff_tra = transform3d(global_csys, new_csys, O, [normal, delta])

    # Calculate back the transformed coordinate diff_tra_norm, which is defined in a local coordinate system "new_csys"
    abs_coord = transform3d(new_csys, global_csys, O, [diff_tra])[0] + extrusion_start_p

    # Create a plane object representing the surface of the object representing the basis for revolution
    eqpn = EquationOfPlane(extrusion_start_p, normal=xvec1, yvec=normal)
    plane_part = make_eq_plane_object(plane_name, eqpn, plane=Plane.XY)
    if make_revolve_p_as_vector:
        ori_vec_model = make_ori_vector(name=f'rotation_axis_center_{plane_name}', origin=abs_coord, csys=new_csys)
        plane_part.add_part(ori_vec_model)
    else:
        plane_part.add_shape(ada.PrimSphere(f'rotation_axis_center_{plane_name}', abs_coord, radius=0.03))

    if eqpn.is_point_in_plane(abs_coord) is False:
        print(f'Point for "{plane_name}" is not in XY-plane')
        diff = abs_coord - extrusion_start_p
        dist = diff.dot(eqpn.normal)
        projected_point = abs_coord - dist*eqpn.normal
        # new_p = transform3d(global_csys, new_csys, O, [projected_point-extrusion_start_p])[0]
        # print(new_p, diff_tra)
        plane_part.add_shape(ada.PrimSphere(f'projected_point_{plane_name}', projected_point, radius=0.03))

    a.add_part(plane_part, overwrite=True)

    return SegData(eqpn, extrusion_start_p, cd, abs_coord)

In [ ]:
# Create a visualization of the 3 planes and axes that make up the basis for each revolved geometry operation

a = pipe_w_multiple_bends()
pipe = list(a.get_all_physical_objects())[0]
pipe_bends = list(filter(lambda x: isinstance(x, ada.PipeSegElbow), pipe.segments))

in_plane_pipe_bend = pipe_bends[0]
out_of_plane_pipe_bend = pipe_bends[1]
out_of_plane_pipe_bend2 = pipe_bends[2]

in_plane_data = evaluate_segment(in_plane_pipe_bend, "InPlane")
out_of_plane1_data = evaluate_segment(out_of_plane_pipe_bend, "OutOfPlane1")
out_of_plane2_data = evaluate_segment(out_of_plane_pipe_bend2, "OutOfPlane2")

a.to_ifc('temp/test.ifc')

In [ ]:
# Visualize rotation points and XY-planes of the revolve surfaces in your jupyter notebook
a

## In-plane example

In [ ]:
curr_data = in_plane_data

diff = curr_data.abs_coord - curr_data.extr_start_point
print(diff, curr_data.eqp.normal)
diff.dot(curr_data.eqp.normal)

## Out-of-plane no.1 example

In [ ]:
curr_data = out_of_plane1_data

diff = curr_data.abs_coord - curr_data.extr_start_point
print(diff, curr_data.eqp.normal)
is_point_on_plane = curr_data.eqp.is_point_in_plane(curr_data.abs_coord)
print(is_point_on_plane)
if is_point_on_plane is False:
    print('Point is not in XY-plane')
    dist = diff.dot(curr_data.eqp.normal)
    projected_point = curr_data.abs_coord - dist*curr_data.eqp.normal
    print(projected_point)

## Out-of-plane no.2 example

In [ ]:
curr_data = out_of_plane2_data

diff = curr_data.abs_coord - curr_data.extr_start_point
print(diff, curr_data.eqp.normal)
curr_data.eqp.is_point_in_plane(curr_data.arc_curve.center)
diff.dot(curr_data.eqp.normal)